In [1]:
import numpy as np

In [21]:
def fit(x_train,y_train):
    result={}
    class_value=set(y_train)
    for current_class in class_value:
        result[current_class]={}
        result["total_data"]=len(y_train)
        current_class_rows=(y_train==current_class)
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        num_features=x_train.shape[1]
        result[current_class]["Total_count"]=len(y_train_current)
        for j in range(1,num_features + 1):
            result[current_class][j]={}
            all_possible_values = set(x_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (x_train_current[:,j-1]==current_value).sum()
    return result
                
            
            

In [28]:
def probability(dictionary,x,current_class):
    output=np.log(dictionary[current_class]["Total_count"])-np.log(dictionary["total_data"])
    num_feature=len(dictionary[current_class].keys())-1
    for j in range(1,num_feature + 1):
        xj=x[j-1]
        count_current_class_value_with_xj=dictionary[current_class][j][xj]+1
        count_current_class=dictionary[current_class]["Total_count"]+ len(dictionary[current_class][j].keys())
        current_xj_pro=np.log(count_current_class_value_with_xj)-np.log(count_current_class)
        output=output + current_xj_pro
    return output

In [23]:
def preForSinglePoint(dictionary,x):
    classes=dictionary.keys()
    best_p=-100
    best_class=-1
    first_run=True
    for current_class in classes:
        if(current_class=="total_data"):
            continue
        p_current_class=probability(dictionary,x,current_class)
        if(first_run or p_current_class > best_p):
            best_p=p_current_class
            best_class=current_class
        first_run=False
    return best_class
            

In [24]:
def predict(dictionary,x_test):
    y_pre=[]
    for x in x_test:
        x_class=preForSinglePoint(dictionary,x)
        y_pre.append(x_class)
    return y_pre

In [25]:
def makelable(column):
    second_limit=column.mean()
    first_limit=0.5*second_limit
    third_limt=1.5* second_limit
    for i in range(0,len(column)):
        if column[i]<first_limit:
            column[i]=0
        elif column[i]<second_limit:
            column[i]=1
        elif column[i]<third_limt:
            column[i]=2
        else:
            column[i]=3
    return column

In [14]:
from sklearn import datasets
data = datasets.load_iris()

In [15]:
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [16]:
x=data.data
y=data.target

In [18]:
for i in range(0,x.shape[-1]):
    x[:,i]=makelable(x[:,i])

In [19]:
from sklearn import model_selection
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,random_state=0)

In [26]:
dictionary=fit(x_train,y_train)

In [29]:
y_pre=predict(dictionary,x_test)

In [30]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pre))
print(confusion_matrix(y_test,y_pre))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

   micro avg       0.97      0.97      0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
